In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
nltk.download("all")
from nltk import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from contractions import CONTRACTION_MAP
import string
import re

TOPWORDS= 5000

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/nbuser/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package

[nltk_data]    |   Unzipping corpora/sentiwordnet.zip.
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Unzipping corpora/sentence_polarity.zip.
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Unzipping corpora/shakespeare.zip.
[nltk_data]    | Downloading package sinica_treebank to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Unzipping corpora/sinica_treebank.zip.
[nltk_data]    | Downloading package smultron to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Unzipping corpora/smultron.zip.
[nltk_data]    | Downloading package state_union to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Unzipping corpora/state_union.zip.
[nltk_data]    | Downloading package stopwords to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Unzipping corpora/stopwords.zip.
[nltk_data]    | 

In [10]:
df = pd.read_csv("train.tsv", sep="\t")
train_text = df["Phrase"]

In [13]:
df.Sentiment.value_counts(normalize=True)

2    0.509945
3    0.210989
1    0.174760
4    0.058990
0    0.045316
Name: Sentiment, dtype: float64

In [15]:
words = []
cleaned_sentences = []
pattern_contraction = re.compile('({})'.format('|'.join(CONTRACTION_MAP.keys())), flags=re.IGNORECASE)

for sentence in train_text.tolist():
    sentence= sentence.lower()
    results_contraction = pattern_contraction.findall(sentence)
    if results_contraction:
        for result in results_contraction:
            sentence = re.sub(result, pattern_contraction[result], sentence)
    
    temp_words = []
    for word in word_tokenize(sentence):
        if word not in string.punctuation:
            words.append(word)
            temp_words.append(word)
    
    cleaned_sentences.append(temp_words)

In [16]:
fd = FreqDist(words)
common_words = set([word for word, _ in fd.most_common(TOPWORDS)])
words2nums = {word:num for num,word in enumerate(common_words)}
words2nums["<unk>"] = len(words2nums)

nums2words = {num:word for num,word in enumerate(common_words)}
nums2words[len(nums2words)] = "<unk>"

In [17]:
train_sentences = []
for sentence in cleaned_sentences:
    temp_list = []
    for word in sentence:
        if word in common_words:
            temp_list.append(words2nums[word])
        else:
            temp_list.append(words2nums["<unk>"])
    train_sentences.append(temp_list)

In [18]:
pad_train_sentences = pad_sequences(train_sentences, maxlen=55)
labels = to_categorical(df["Sentiment"].tolist())

In [54]:
model = Sequential()
e = Embedding(TOPWORDS+1, 300, input_length=55)
model.add(e)
model.add(LSTM(20))
#model.add(Dense(100, activation="relu"))
model.add(Dense(5, activation="softmax"))
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['acc'])

In [57]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 55, 300)           1500300   
_________________________________________________________________
lstm_6 (LSTM)                (None, 20)                25680     
_________________________________________________________________
dense_16 (Dense)             (None, 5)                 105       
Total params: 1,526,085
Trainable params: 1,526,085
Non-trainable params: 0
_________________________________________________________________
None


In [58]:
model.fit(pad_train_sentences, labels, epochs=1, verbose=3, batch_size=128)

Epoch 1/1


KeyboardInterrupt: 